# PCA Parameters
<b>n_components</b> : Number of components to keep.<br>
Use Minka's MLE if n_components='mle; & svd_solver=='full' to guess dimension.


<b>copy</b> : Make copy of data, so it is not overwritten with fit. Use fit_transform otherwise.

<b>whiten</b> : If true, components_ vectors are multiplied by sqrt(n_samples) and divided by singular values to ensure uncorrelated outputs with unit component--wise vectors.<br>
<i>Warning:</i> Removes info, but can imporve preditive accuracy by making data respect hard-wired assumptions

<b>svd_solver</b> : Singular Value Decomposition (Lower Dim. Projection Technique)

<i>svd_solver</i>=='auto': Default policy based on X.shape and n_components. Randomized if large and n_components is < 80%. Else Full

<i>svd_solver=='full'</i> : Number of components such that the amount of variance that needs to be explained is greater than % specified.

<i>svd_solver=='randomized'</i> : Halko Method

<b>iterated_power</b> : Number of iterations for power method computed by svd_solver == 'randomized'.

<b>random_state</b> : Random Seed used by randomized.

# PCA Attributes
<b>components_</b> : Principal axes in feature space, represents directions of maximum variance in data.

<b>explained__variance__</b> : Amount of variance explained by each of the selected components.

<b>explained__variance_ratio__</b> : Percent of variance explained by each selected components.

<b>mean_</b> : Estimated number of components. When set to mle or 0 < n_components < 1, number is estimated from input data. Otherwise it equals parameter n_components.

<b>noise_variance</b> : Estimated noise covariance from Bishop PCA.


In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
# Interested in Feature space with > 99% Variance
